In [1]:
import bs4
import yaml
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
with open("config.yaml", 'r') as stream:
    config=yaml.safe_load(stream)
    
metting_id = config["data"]["metting_id"]
zoomfolderpath = config["folders"]["zoomfolderpath"]
matchfolderpath = config["folders"]["matchfolderpath"]

email = config["login"]["email"]
password = config["login"]["pass"]

In [3]:
"""
Selecting zoom names
"""
df = pd.read_csv(f"{zoomfolderpath}/participants_{metting_id}.csv")

zoom_names = set(
    df.rename(columns={"Name (Original Name)":"nomes"})
      .drop_duplicates(subset="nomes")
      .loc[:,"nomes"]
      .str.lower()
      .str.strip()
      .str.split()
      .apply(lambda x: x[0])
)

In [4]:
#driver = webdriver.Chrome('chromedriver')

driver = webdriver.Chrome(r'/home/rocabrera/.local/share/chromedriver')
driver.get("https://class.letscode.com.br/")

/tmp/ipykernel_53669/931396387.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/home/rocabrera/.local/share/chromedriver')


In [5]:
"""
Handle Login 
"""

email_element = driver.find_element(by=By.ID, value="mat-input-1")
email_element.send_keys(email)

pass_element = driver.find_element(by=By.ID, value="mat-input-2")
pass_element.send_keys(password)

login_button = driver.find_element(by=By.XPATH, value='//*[@id="login-form"]/form/button')
login_button.click()

In [6]:
"""
Essa parte demora ... tem que ter um algum tipo de await complete aq
"""

teachers_corner_drop_xpath = '//*[@id="container-1"]/fuse-sidebar/navbar/navbar-vertical-style-1/div[2]/div[2]/fuse-navigation/div/fuse-nav-vertical-group[1]/div[2]/fuse-nav-vertical-collapsable/a'
teachers_corner_xpath = '//*[@id="container-1"]/fuse-sidebar/navbar/navbar-vertical-style-1/div[2]/div[2]/fuse-navigation/div/fuse-nav-vertical-group[1]/div[2]/fuse-nav-vertical-collapsable/div/fuse-nav-vertical-item[3]/a'
xpaths = [teachers_corner_drop_xpath, teachers_corner_xpath]

for xpath in xpaths:
    # elemento = EC.presence_of_element_located((By.XPATH, xpath))
    # print(elemento)
    time.sleep(1)
    # WebDriverWait(driver, 30).until(elemento) #This is a dummy element)
    driver.find_element(by=By.XPATH, value=xpath).click()


NAO TEM LOGICA PARA SELECIONAR O PATH DA DATA

In [7]:
aba_presencas = '//*[@id="mat-tab-label-1-3"]'
driver.find_element(by=By.XPATH, value=aba_presencas).click()

In [8]:
turma_xpath = '//*[@id="mat-select-5"]/div/div[1]/span/mat-select-trigger'
driver.find_element(by=By.XPATH, value=turma_xpath).click()

In [9]:
turma_number_xpath = '//*[@id="mat-option-34"]'
driver.find_element(by=By.XPATH, value=turma_number_xpath).click()

In [10]:
data_xpath = '//*[@id="widget-class-checkin"]/div/div/div[2]/div[1]/mat-list/mat-list-item[123]/div'
driver.find_element(by=By.XPATH, value=data_xpath).click() 

In [11]:
alunos = driver.find_elements(by=By.CLASS_NAME, value="members-table-row")

for aluno in alunos[4:]:
    driver.execute_script('arguments[0].scrollIntoView();', aluno)
    
    name, *_, remote_checkbox = aluno.find_elements(by=By.CLASS_NAME, value="members-table-column") 
    #checkbox = aluno.find_element(by=By.CLASS_NAME, value="mat-checkbox-inner-container") 
    parsed_class_name = name.text.lower().split()[0]
    if parsed_class_name in zoom_names:
        remote_checkbox.click()
    else:
        print(f"Faltou: {parsed_class_name}")
    # webelement[-1].click()
    
    # remoto.click()
    # time.sleep(0.01)


Faltou: daniela
Faltou: eder
Faltou: elias
Faltou: enzo
Faltou: fabio
Faltou: marco
Faltou: ronni
Faltou: leticia
Faltou: sara


In [ ]:
# """
# Da para utilizar para fazer scroll down
# """

# names = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'members-table-row')))

# while True:
#     # Scroll down to last name in list
#     driver.execute_script('arguments[0].scrollIntoView();', hotels[-1])
#     try:
#         # Wait for more names to be loaded
#         WebDriverWait(driver, 15).until(lambda driver: len(wait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'hotel-name')))) > len(hotels))
#         # Update names list 
#         hotels = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'hotel-name')))
#     except:
#         # Break the loop in case no new names loaded after page scrolled down
#         break
